# Random Forest

Definition of a Random Forest
---

### Improving on Decision Trees

The main drawback of decision trees is their tendency to overfit. We can improve their performance with pruning techniques, but this module will introduce a better model. 

Decision trees are very susceptible to random idiosyncrasies in the training dataset. They have very high variance, which means if you randomly select the training dataset, you end up with very different models.

One advantage of decision trees over models like logistic regression is that they make no assumptions about how the data is structured. In logistic regression, we assume to draw a line to split the data points. Sometimes when the data is not structured like that, decision trees can still capture the essence of the data.

This module is about **random forests**. As its name suggests, it is a model built with multiple trees. The goal of random forests is to take advantage of decision trees while mitigating the variance issue.

### Bootstrapping

A **bootstrapping** sample are data points randomly selected from our original dataset with replacement.